In [2]:
import quandl
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split



In [4]:
#https://medium.com/@randerson112358/predict-stock-prices-using-python-machine-learning-53aa024da20a
# Get the stock data

quandl.ApiConfig.api_key = 'qKYyca8_q3vn5ws7FpwZ'

#df = quandl.get("WIKI/FB", trim_start = "2012-03-15", trim_end ="2018-03-15")

# Take a look at the data
#df.head()

df = quandl.get_table('WIKI/PRICES', ticker = ['AAPL', 'MSFT', 'WMT',"FB"], 
                        qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                        date = { 'gte': '2012-03-15', 'lte': '2018-03-15' }, 
                        paginate=True)
df.head

<bound method NDFrame.head of      ticker       date   adj_close
None                              
0       WMT 2018-03-15   87.510000
1       WMT 2018-03-14   87.670000
2       WMT 2018-03-13   88.300000
3       WMT 2018-03-12   88.070000
4       WMT 2018-03-09   88.720000
5       WMT 2018-03-08   87.920000
6       WMT 2018-03-07   87.740000
7       WMT 2018-03-06   89.060000
8       WMT 2018-03-05   89.980000
9       WMT 2018-03-02   88.770000
10      WMT 2018-03-01   89.080000
11      WMT 2018-02-28   90.010000
12      WMT 2018-02-27   91.520000
13      WMT 2018-02-26   93.120000
14      WMT 2018-02-23   92.890000
15      WMT 2018-02-22   92.770000
16      WMT 2018-02-21   91.520000
17      WMT 2018-02-20   94.110000
18      WMT 2018-02-16  104.780000
19      WMT 2018-02-15  103.230000
20      WMT 2018-02-14  101.700000
21      WMT 2018-02-13  100.980000
22      WMT 2018-02-12   99.550000
23      WMT 2018-02-09   99.370000
24      WMT 2018-02-08  100.020000
25      WMT 2018-02-07  1

In [33]:
# Get the Adjusted Close Price 
df = df[['Adj. Close']] 
# Take a look at the new data 
df.head()

,Adj. Close
Date,
2012-05-18,38.2318
2012-05-21,34.0300
2012-05-22,31.0000
2012-05-23,32.0000
2012-05-24,33.0300


In [34]:
# A variable for predicting 'n' days out into the future
forecast_out = 60
#'n=30' days
#Create another column (the target ) shifted 'n' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
#print the 12)
df.tail(70)

,Adj. Close,Prediction
Date,,
2017-12-04,171.4700,176.62
2017-12-05,172.8300,180.40
2017-12-06,176.0600,179.78
2017-12-07,180.1400,183.71
2017-12-08,179.0000,182.34
2017-12-11,179.0400,185.23
2017-12-12,176.9600,184.76
2017-12-13,178.3000,181.88
2017-12-14,178.3900,184.19


In [35]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

#Remove the last 'n' rows
X = X[:-forecast_out]
print(X)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [178.3   ]
 [178.39  ]
 [180.18  ]]


In [36]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array 
y = np.array(df['Prediction'])
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]
print(y)

[ 20.38  21.2   19.87 ... 181.88 184.19 183.86]


In [37]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [38]:
# Create and train the Support Vector Machine (Regressor) 
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1) 
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [39]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9792415647324464


In [40]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [41]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9733333610682646


In [42]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[180.82  ]
 [179.51  ]
 [177.89  ]
 [177.45  ]
 [177.2   ]
 [175.99  ]
 [177.62  ]
 [177.92  ]
 [176.46  ]
 [181.42  ]
 [184.67  ]
 [184.33  ]
 [186.85  ]
 [188.28  ]
 [187.87  ]
 [187.84  ]
 [187.77  ]
 [179.37  ]
 [178.39  ]
 [177.6   ]
 [179.8   ]
 [181.29  ]
 [185.37  ]
 [189.35  ]
 [186.55  ]
 [187.48  ]
 [190.    ]
 [185.98  ]
 [187.12  ]
 [186.89  ]
 [193.09  ]
 [190.28  ]
 [182.65  ]
 [185.31  ]
 [180.18  ]
 [171.5499]
 [175.98  ]
 [176.41  ]
 [173.15  ]
 [179.52  ]
 [179.96  ]
 [177.36  ]
 [176.01  ]
 [177.91  ]
 [178.99  ]
 [183.29  ]
 [184.93  ]
 [181.46  ]
 [178.32  ]
 [175.94  ]
 [176.62  ]
 [180.4   ]
 [179.78  ]
 [183.71  ]
 [182.34  ]
 [185.23  ]
 [184.76  ]
 [181.88  ]
 [184.19  ]
 [183.86  ]]


In [43]:
# Print linear regression model predictions for the next '30' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[190.60849659 189.25718805 187.58610421 187.13222959 186.87434628
 185.62619106 187.30759024 187.61705021 186.11101168 191.22741653
 194.57989955 194.22917825 196.828642   198.30373453 197.8808059
 197.84985991 197.77765258 189.1127734  188.10187083 187.28695957
 189.55633269 191.09331721 195.30197281 199.40747509 196.51918203
 197.47850794 200.0779717  195.93120809 197.10715598 196.86990333
 203.26540939 200.366801   192.49620241 195.24008082 189.94831532
 181.04608034 185.61587573 186.05943502 182.69663667 189.26750338
 189.72137801 187.03939159 185.64682173 187.60673487 188.72079077
 193.15638368 194.84809819 191.26867786 188.0296635  185.5746144
 186.276057   190.17525263 189.53570203 193.58962764 192.17642711
 195.15755816 194.67273754 191.70192182 194.0847636  193.74435763]


In [44]:
# Print support vector regressor model predictions for the next '30' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[180.39738039 184.04275158 178.17758101 178.0866333  178.38854766
 179.63860772 178.01076547 178.22009652 179.56415413 177.02962797
 188.34781314 190.34489775 143.70265737 119.51425244 124.64979712
 125.0825016  126.12318055 183.68062653 179.50309201 178.01304347
 184.33985287 177.62397875 178.1972807  111.84329131 150.72043122
 130.8993495  109.88354716 164.64094084 137.84072494 142.80330365
 108.37451136 109.39869007 179.67753092 179.34358899 183.68503592
 180.87554535 179.63119023 179.60960958 179.43860142 184.06361263
 184.21041381 178.17331814 179.65231847 178.20537689 182.19470014
 185.7181578  185.46618853 176.87162958 179.24457515 179.5978108
 179.36983177 182.79321516 184.3409313  189.11302311 177.50629518
 180.80186691 187.48020941 176.1155072  190.58422762 189.91523854]
